Data loader for noise0,0.1,0.2

In [ ]:
import os
import collections
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from statistics import mean



PRECISION = 0.001

SYSTEMS = ['popper_acc', 'popper_accminsize', 'popper_lexfn', 'popper_lexfnsize', 'popper_lexfp', 'popper_lexfpsize', 'popper_mdl']

TRIALS = [i for i in range(2)]
STEP_NPROGS = 1000
MAX_NPROGS = 80000


def std_err(lst):
    data = np.array(lst)
    return np.std(data, ddof=1) / np.sqrt(np.size(data))


def read_results(result_file):
    with open(result_file, 'r') as f:
        for line in f.readlines():
            if line.startswith('accuracy:'):
                acc = float(line.split(':')[1])
    return acc


def read_data(DOMAIN,TASKS):
    data_accuracy = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(list)))
    stats = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(dict)))
    for sys in SYSTEMS:
        for task in TASKS:
            for trial in TRIALS:
                print(sys, task, trial)
                output_path = os.path.join('/content/drive/MyDrive/popper/plot_timeout/experimental_data', DOMAIN, f'{task}', sys, str(trial), "nprogs")
                print(output_path)
                dirs = os.listdir(output_path)
                n_progs = sorted([int(d.split(".")[0]) for d in dirs if not d.startswith('.')])
                k, current_key = 0, 0
                while k < MAX_NPROGS:
                    if n_progs[current_key] < k and current_key < len(n_progs) - 1:
                        current_key += 1
                    data_accuracy[sys][task][k].append(read_results(os.path.join(output_path, f"{n_progs[current_key]}.pl")))
                    k += STEP_NPROGS
                    if k > max(n_progs):
                      print(current_key, len(n_progs),n_progs)
                      break  # Break the loop if no further data is available
            acc_av, acc_sem = [], []
            for k in data_accuracy[sys][task]:
                acc_av.append(mean(data_accuracy[sys][task][k]))
                acc_sem.append(std_err(data_accuracy[sys][task][k]))
            stats[sys][task]['timeout'] = sorted(data_accuracy[sys][task].keys())
            stats[sys][task]['acc_av'] = acc_av
            stats[sys][task]['acc_sem'] = acc_sem
    return stats


def load_data(domain, noise_levels, prefixes):
    all_data = []
    for noise_level in noise_levels:
        tasks = [f"{prefix}__{noise_level}" for prefix in prefixes]
        data = read_data(domain, tasks)
        all_data.append(data)
    return all_data



In [ ]:
def calculate_accuracy_differences(data_list):

    diff_data_list = []

    for stats in data_list:
        diff_data = calculate_accuracy_difference(stats)
        diff_data_list.append(diff_data)

    return diff_data_list

def calculate_accuracy_difference(stats):

    accuracy_diff = {}
    stats = stats[0]
    for sys in stats:
        accuracy_diff[sys] = {}
        for task in stats[sys]:
            print(task)
            accuracies = stats[sys][task]['acc_av']  # List of accuracy averages for different timeout values
            max_accuracy = max(accuracies)
            last_accuracy = accuracies[-1]
            accuracy_diff[sys][task] = max_accuracy - last_accuracy

    return accuracy_diff




In [ ]:


def plot_average_diff(data_0_diff, data_1_diff, data_2_diff, systems, noise_levels=[0, 0.1, 0.2]):

    # Initialize a dictionary to store average diffs for each system
    avg_diff_per_system = {system: [] for system in systems}

    # Combine the dictionaries into a list for easier iteration
    data_list = [data_0_diff, data_1_diff, data_2_diff]

    # Calculate the average difference for each system at each noise level
    for data in data_list:
        for system in systems:
            if system in data:
                diffs = list(data[system].values())
                avg_diff = sum(diffs) / len(diffs) if diffs else 0
                avg_diff_per_system[system].append(avg_diff)
            else:
                avg_diff_per_system[system].append(0)  # If system data is missing, append 0

    # Plotting
    plt.figure(figsize=(10, 6))
    for system in systems:
        plt.plot(noise_levels, avg_diff_per_system[system], marker='o', label=system)

    plt.title('Average Accuracy Difference vs Noise Levels')
    plt.xlabel('Noise Level')
    plt.ylabel('Average Accuracy Difference')
    plt.xticks(noise_levels)
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:

def plot_accuracy_difference(data_0_diff, data_1_diff, data_2_diff, systems, noise_levels=[0.0, 0.1, 0.2]):


    def average_every_seven(nums):

        averages = []
        for i in range(0, len(nums), 7):
            chunk = nums[i:i+7]
            avg = sum(chunk) / len(chunk)
            averages.append(avg)
        return averages

    # Combine data dictionaries into a list
    data_list = [data_0_diff, data_1_diff, data_2_diff]
    diff_system = []

    # Calculate the average difference for each system across the data_list
    for i in data_list:
        for system in systems:
            if system in i:
                diffs = list(i[system].values())
                avg_diff = sum(diffs) / len(diffs) if diffs else 0
                diff_system.append(avg_diff)

    # Calculate the average difference for every 7 systems
    av_acc = average_every_seven(diff_system)

    # Plotting the results
    if len(noise_levels) != len(av_acc):
        raise ValueError("Length of noise_levels and av_acc must be the same")

    plt.figure(figsize=(8, 5))
    plt.plot(noise_levels, av_acc, marker='o', linestyle='-', color='b', label='Accuracy Difference')

    plt.title('Average Accuracy Difference vs Noise Levels')
    plt.xlabel('Noise Level')
    plt.ylabel('Average Accuracy Difference')
    plt.xticks(noise_levels)
    plt.grid(True)
    plt.legend()
    plt.show()




In [ ]:
SYSTEMS = ['popper_acc', 'popper_accminsize', 'popper_lexfn', 'popper_lexfnsize', 'popper_lexfp', 'popper_lexfpsize', 'popper_mdl']



data_0 = load_data('zendo', ['0'], ['zendo1', 'zendo2', 'zendo3', 'zendo4'])
data_1 = load_data('zendo_noise', ['0.1'], ['zendo1', 'zendo2', 'zendo3', 'zendo4'])
data_2 = load_data('zendo_noise', ['0.2'], ['zendo1', 'zendo2', 'zendo3', 'zendo4'])
data_0_diff, data_1_diff, data_2_diff = calculate_accuracy_differences([data_0, data_1, data_2])
#print(data_0_diff['popper_acc'])
# plot noise vs diff per system
plot_average_diff(data_0_diff, data_1_diff, data_2_diff, SYSTEMS)
# noise vs diff
plot_accuracy_difference(data_0_diff, data_1_diff, data_2_diff, SYSTEMS)


data_0 = load_data('iggp', ['0.0'], ['iggp-attrition', 'iggp-centipede', 'iggp-coins-goal', 'iggp-coins-next','iggp-md','iggp-rps-next'])
data_1 = load_data('iggp', ['0.1'], ['iggp-attrition', 'iggp-centipede', 'iggp-coins-goal', 'iggp-coins-next','iggp-md','iggp-rps-next'])
data_2 = load_data('iggp', ['0.2'], ['iggp-attrition', 'iggp-centipede', 'iggp-coins-goal', 'iggp-coins-next','iggp-md','iggp-rps-next'])
data_0_diff, data_1_diff, data_2_diff = calculate_accuracy_differences([data_0, data_1, data_2])
#print(data_0_diff['popper_acc'])
plot_average_diff(data_0_diff, data_1_diff, data_2_diff, SYSTEMS)